## format and split data

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
train = pd.read_csv('train_small600.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Droping the nan values in the type column to isolate the data that we had marked as train data (rows that we'd manually filled)
train.dropna(subset=['type'], inplace=True)

In [4]:
#Defining the test data
test = pd.read_csv('labeled_output.csv')

In [5]:
test.dropna(subset=['text'], inplace=True)

In [6]:
train.head()

,channel_type,channel_name,id,date,from,text,from_2,type,category
0,personal_chat,TZ155 Nontobeko Mthembu,226915,2018-03-22T17:59:29,Setup (Master),Sawubona Jessie 😊. ;;We can see that you are e...,setup,noncontent,setup
1,private_group,19:00 (10/07) Thabiso (2)🏆3️⃣,376217,2018-04-03T19:01:47,ThishaBot,Any questions?,bot,noncontent,conv
2,private_group,20:00 (11/07) Irfaan (2)🏆3️⃣,873757,2018-05-15T20:52:52,Irfaan Moolla,Today,student,noncontent,conv
3,private_group,13:00 (10/07) Fatima (3)🏆3️⃣,1368973,2018-07-02T19:05:13,TZ Simphiwe Mfaba,Kulungile 💪,tutor,noncontent,conv
4,private_group,20:00 (09/07) Jenna (1)🏆4⃣,1234119,2018-06-18T20:12:31,ThishaBot,"Surprisingly, the patient seems satisfied with...",bot,content,non mcq


In [7]:
test.drop(columns=['text_type'], inplace=True)

In [8]:
#Defining the training data
X = train['text']
y = train['type']

In [9]:
print(X.shape)
print(y.shape)

(604,)
(604,)


In [10]:
#Splitting the data into a train and test set
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=128, test_size=0.2)

In [1]:
#Label encoding the data to make each unique value as an integer because the neural network cannot predict a string, it can only predict a number
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

NameError: name 'y_train' is not defined

In [12]:
#Vectorizing the data so that we can put it through the neural network later
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vec = TfidfVectorizer(analyzer='word',
                         min_df=1,
                         strip_accents='unicode',
                         token_pattern=r'\w{1,}',
                         ngram_range=(2,3),
                         )
x_train_vec = tf_vec.fit_transform(x_train)
x_test_vec = tf_vec.transform(x_test)
print (x_train_vec.shape)
print (x_test_vec.shape)

(483, 5020)
(121, 5020)


## neural net

In [13]:
from keras import models
from keras import layers
from keras.layers import Dropout
from keras import optimizers
from keras.optimizers import SGD

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
# def create_model():
#     inputs = layers.Input(shape=(2163,))
#     hidden_1 = layers.Dense(units=16, activation='relu')(inputs)
#     dropout1 = layers.Dropout(0.2)(hidden_1)
#     hidden_2 = layers.Dense(units=16, activation='relu')(hidden_1)
#     dropout_2 = layers.Dropout(0.2)(hidden_2)
#     outputs = layers.Dense(1, activation='softmax')(hidden_2)

#     model= models.Model(inputs=inputs, outputs=outputs)
#     #model.summary()
#     model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='Adam')
#     return model

## parameter training
Creating a model to find the best parameters for our neural network. We used a gridsearch to do this. This will have to be run on an instance

In [15]:
from keras import Sequential
from keras.layers import Dense
from keras.losses import binary_crossentropy

In [16]:
def create_model_2(hidden_layers=1,dropout_rate=0):
    model2 = Sequential()
    
    model2.add(Dense(16, input_shape=(2163,), activation='relu',))
    for i in range(hidden_layers):
        model2.add(Dense(16, activation='relu'))
        model2.add(Dropout(dropout_rate))
    model2.add(Dense(1, activation='sigmoid'))

    model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model2

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier

model2 = KerasClassifier(build_fn=create_model_2, verbose=0, epochs=32, batch_size=8, shuffle=True)

In [18]:
from sklearn.model_selection import GridSearchCV

hidden_layers= [2,4,6,8,10,12]
dropout_rate=[0.2,0.4,0.6,0.8]


param_grid2 = dict(hidden_layers=hidden_layers, dropout_rate=dropout_rate)

grid_2 = GridSearchCV(estimator=model2, param_grid=param_grid2, n_jobs=-1)

In [19]:
#grid2_pred = grid_2.fit(x_train_vec, y_train)

In [20]:
#print(grid2_pred.best_score_)
#print(grid2_pred.best_params_)

In [21]:
def create_model_3(neurons):
    model3 = Sequential()
    
    model3.add(Dense(neurons, input_shape=(2163,), activation='relu',))
    for i in range(4):
        model3.add(Dense(neurons, activation='relu'))
        model3.add(Dropout(0.2))
    model3.add(Dense(1, activation='sigmoid'))

    model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model3

In [22]:
model3 = KerasClassifier(build_fn=create_model_3, verbose=0, epochs=32, batch_size=8, shuffle=True)

In [23]:
neurons= [4,8,16,32,64]

param_grid3 = dict(neurons=neurons)

grid_3 = GridSearchCV(estimator=model3, param_grid=param_grid3, n_jobs=-1)

In [24]:
# grid3_pred = grid_3.fit(x_train_vec, y_train)

In [25]:
# print(grid3_pred.best_score_)
# print(grid3_pred.best_params_)

## Neural Network

In [28]:
#Defining the layers of the neural network
inputs = layers.Input(shape=(5020,))
hidden_1 = layers.Dense(units=16, activation='relu')(inputs)
dropout1 = layers.Dropout(0.2)(hidden_1)
hidden_2 = layers.Dense(units=16, activation='relu')(hidden_1)
dropout2 = layers.Dropout(0.2)(hidden_2)
hidden_3=layers.Dense(units=16,activation='relu')(hidden_2)
dropout3=layers.Dropout(0.2)(hidden_3)
hidden_4=layers.Dense(units=16,activation='relu')(hidden_3)
dropout4=layers.Dropout(0.2)(hidden_4)
hidden_5=layers.Dense(units=16,activation='relu')(hidden_4)
dropout5=layers.Dropout(0.2)(hidden_5)
hidden_6=layers.Dense(units=16,activation='relu')(hidden_5)
dropout6=layers.Dropout(0.2)(hidden_6)
hidden_7=layers.Dense(units=16,activation='relu')(hidden_6)
dropout7=layers.Dropout(0.2)(hidden_7)
hidden_8=layers.Dense(units=16,activation='relu')(hidden_7)
dropout8=layers.Dropout(0.2)(hidden_8)
outputs = layers.Dense(1, activation='sigmoid')(hidden_8)

network= models.Model(inputs=inputs, outputs=outputs)
network.summary()
network.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='Adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5020)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                80336     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
__________

In [29]:
#Testing the accuracy of the network on the test data
network.fit(x=x_train_vec, y=y_train, batch_size=16, epochs=30, verbose=0)
test_loss, test_acc = network.evaluate(x_test_vec, y_test)
print (test_loss)
print (test_acc)

121/121 [==============================] - 0s 613us/step
0.26210343061892455
0.9173553674674231
